## Finetune PaliGemma for Object Detection 

#### CREATE PALIGEMMA STYLE DATASET

In [ ]:
from pg_datasets import coco_to_xyxy, show_random_elements, string2list, resize_bbox, coco_to_xyxy, convert_to_detection_string

In [189]:
from huggingface_hub import notebook_login

notebook_login()

In [190]:
from datasets import load_dataset

xray_dataset = load_dataset('dmusingu/object-detection-chest-x-ray')

README.md:   0%|          | 0.00/968 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/50.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/6.42M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/6.88M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/736 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/93 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/98 [00:00<?, ? examples/s]

In [191]:
train_dataset = xray_dataset['train']
valid_dataset = xray_dataset['validation']
test_dataset = xray_dataset['test']

In [192]:
show_random_elements(train_dataset.remove_columns(['image']), num_examples=2)

,id,image_id,category_id,bbox,area,segmentation,iscrowd,license,height,width
0,284,269,Necklace,"[283, 1, 537.5, 595.5]",320081.25,[],0,1,1024,1024
1,666,627,Vascular Port,"[160, 435, 85, 102.5]",8712.50,[],0,1,1024,1024


In [193]:
train_dataset = train_dataset.map(lambda x: {'bbox': string2list(x['bbox'])})
valid_dataset = valid_dataset.map(lambda x: {'bbox': string2list(x['bbox'])})
test_dataset = test_dataset.map(lambda x: {'bbox': string2list(x['bbox'])})

Map:   0%|          | 0/736 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [194]:
train_dataset = train_dataset.map(resize_bbox)
valid_dataset = valid_dataset.map(resize_bbox)
test_dataset = test_dataset.map(resize_bbox)

Map:   0%|          | 0/736 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [195]:
train_dataset = train_dataset.map(coco_to_xyxy)
valid_dataset = valid_dataset.map(coco_to_xyxy)
test_dataset = test_dataset.map(coco_to_xyxy)

Map:   0%|          | 0/736 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [196]:
train_dataset = train_dataset.map(convert_to_detection_string)
valid_dataset = valid_dataset.map(convert_to_detection_string)
test_dataset = test_dataset.map(convert_to_detection_string)

Map:   0%|          | 0/736 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [198]:
# create a paligemma prefix
dataset_objects = list(set(train_dataset['train']['category_id']))
dataset_objects

['Necklace', 'Shoulder Endoprosthesis', 'ICD', 'Vascular Port']

In [199]:
prefix = 'detect ' + "; ".join(dataset_objects)
prefix

'detect Necklace; Shoulder Endoprosthesis; ICD; Vascular Port'

In [200]:
train_prefix = [prefix] * len(train_dataset)
valid_prefix = [prefix] * len(valid_dataset)
test_prefix = [prefix] * len(test_dataset)

In [201]:
train_dataset = train_dataset.add_column('prefix', train_prefix)
valid_dataset = valid_dataset.add_column('prefix', valid_prefix)
test_dataset = test_dataset.add_column('prefix', test_prefix)

In [202]:
show_random_elements(train_dataset.remove_columns(['image']), num_examples=2)

,id,image_id,category_id,bbox,area,segmentation,iscrowd,license,height,width,resized_bbox,xyxy,suffix,prefix
0,731,688,Vascular Port,"[278.0, 280.0, 92.5, 115.0]",10637.5,[],0,1,1024,1024,"[60, 61, 20, 25]","[60, 61, 80, 86]",<loc0061><loc0060><loc0086><loc0080> Vascular Port,detect Necklace; Shoulder Endoprosthesis; ICD; Vascular Port
1,210,201,Shoulder Endoprosthesis,"[1.0, 163.0, 156.0, 219.0]",34164.0,[],0,1,1024,1024,"[0, 35, 34, 47]","[0, 35, 34, 82]",<loc0035><loc0000><loc0082><loc0034> Shoulder Endoprosthesis,detect Necklace; Shoulder Endoprosthesis; ICD; Vascular Port
